In [ ]:
# Analyse diff STAR HHV8 :
# install :

In [1]:
import os
import pandas as pd

count_dir = "./count_star"
output_matrix = "./count_star/gene_count_matrix_STAR.csv"

files = [f for f in os.listdir(count_dir) if f.endswith("_ReadsPerGene.out.tab")]
dfs = []

for file in files:
    sample_name = file.replace("_not_hg38_ReadsPerGene.out.tab", "")  
    path = os.path.join(count_dir, file)


    df = pd.read_csv(path, sep='\t', skiprows=1, header=None)


    if len(df.columns) < 2:
        logging.warning(f"Fichier {file} a seulement {len(df.columns)} colonnes → skip")
        continue

    # Geneid + dernier colonne = nmb de reads
    df.columns = ['Geneid'] + [f"col{i}" for i in range(1, len(df.columns))]
    df = df[['Geneid', df.columns[-1]]]
    df.columns = ['Geneid', sample_name]

    dfs.append(df.set_index('Geneid'))

combined_df = pd.concat(dfs, axis=1)

# rm les  "unassigned"
gene_mask = ~combined_df.index.str.contains("N_", na=True) & ~combined_df.index.str.contains("unassigned", na=True)
combined_df = combined_df[gene_mask]


combined_df.to_csv(output_matrix)
print(f"Matrice Done  : {output_matrix}")

Matrice Done  : ./count_star/gene_count_matrix_STAR.csv


In [ ]:
#Charger les data et librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


matrix_path = "./count_star/gene_count_matrix_STAR.csv"
df_counts = pd.read_csv(matrix_path, index_col=0)

print("Dimensions:", df_counts.shape)
df_counts.head()

In [ ]:
#Tableau : Nombre total de reads par échantillon + Graphique

# total de reads par sample
total_reads_per_sample = df_counts.sum().to_frame(name="Total Reads")
total_reads_per_sample["Moyenne par gène"] = df_counts.mean()
total_reads_per_sample["Médiane par gène"] = df_counts.median()

print("Statistiques par échantillon")
display(total_reads_per_sample.sort_values(by="Total Reads", ascending=False))

# Histogramme
plt.figure(figsize=(10,6))
sns.barplot(x=total_reads_per_sample.index, y="Total Reads", data=total_reads_per_sample, palette="viridis")
plt.xticks(rotation=90)
plt.title("Nombre total de reads par échantillon")
plt.xlabel("Sample")
plt.ylabel("Total reads")
plt.show()

In [ ]:

# Tableau : Statistiques par échantillon + Boxplot
stats_df = df_counts.describe().T
stats_df = stats_df[['mean', '50%', 'std', 'min', 'max']]
print("Statistiques descriptives par échantillon")
display(stats_df)

# Graphique : Boxplot
plt.figure(figsize=(12,8))
sns.boxplot(data=df_counts.T)
plt.xticks(rotation=90)
plt.title("Distribution des comptages (boxplots) par échantillon")
plt.ylabel("Nombre de reads")
plt.show()

In [ ]:
# Dictionnaire : identifiant → abréviation/fonction
gene_abbr = {
    "HHV8GK18_gp01": "K1",
    "HHV8_gs01": "miscRNA01",
    "HHV8GK18_gp39": "ORF35",
    "HHV8GK18_gp62": "vIRF-1 (K9)",
    "HHV8GK18_gp63": "vIRF-4",
    "HHV8GK18_gp64": "vIRF-3",
    "HHV8GK18_gp65": "vIRF-2",
    "HHV8GK18_gp79": "K12",
    "HHV8_gs02": "miscRNA02",
    "HHV8_gs03": "miscRNA03",
    "HHV8_gs04": "miscRNA04",
    "HHV8_gs05": "miscRNA05",
    "HHV8_gs06": "miscRNA06",
    "HHV8_gs07": "miscRNA07",
    "HHV8_gs08": "miscRNA08",
    "HHV8_gs09": "miscRNA09",
    "HHV8_gs10": "miscRNA10"
}

# Exemple d'utilisation : obtenir l'abréviation d'un gène 

gene_ids = df_counts.index  # Utilise l'index du DataFrame comme liste des gènes
for gene in gene_ids:
    print(f"{gene} : {gene_abbr.get(gene, 'NA')}")

In [ ]:
#Corrélation entre échantillons + Heatmap
corr_matrix = df_counts.corr(method="pearson")
print("🧮 Corrélation entre échantillons (Pearson)")
display(corr_matrix)

# Graphique : Heatmap
plt.figure(figsize=(10,8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Heatmap de corrélation (Pearson) entre échantillons")
plt.show()

In [ ]:
#Filtrage des gènes faiblement exprimés + Stats
# Filtrer les gènes exprimés dans au moins 2 échantillons avec > 10 reads
threshold = 10
min_samples = 2
mask = (df_counts > threshold).sum(axis=1) >= min_samples
filtered_df = df_counts[mask]

print(f"🧬 Gènes avant filtrage : {df_counts.shape[0]}")
print(f"🧬 Gènes après filtrage (>={min_samples} échantillons avec >{threshold} reads) : {filtered_df.shape[0]}")

# Exemple : Afficher quelques gènes filtrés
print("\n🧬 Exemples de gènes conservés :")
display(filtered_df.sample(5))